In [1]:
# add scripts/ folder to path
import os, sys

SCRIPTS_PATH = os.environ['DSX_PROJECT_DIR'] + '/scripts'
sys.path.insert(0, SCRIPTS_PATH)

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import visualization # custom script

## Load Dataset

In [3]:
DATASET_PATH = "/user-home/libraries/text-analytics/datasets/aclImdb-small"
TEST_PATH = DATASET_PATH + "/test/"
TRAIN_PATH = DATASET_PATH + "/train/"

In [4]:
from sklearn.datasets import load_files

reviews_test = load_files(TEST_PATH)
reviews_train = load_files(TRAIN_PATH)

In [5]:
text_test, y_test = reviews_test.data, reviews_test.target

print("Number of documents in test data: {}".format(len(text_test)))
print("Samples per class (test): {}".format(np.bincount(y_test)))

Number of documents in test data: 2000
Samples per class (test): [1000 1000]


In [6]:
text_test = [doc.replace(b"<br />", b" ").decode('utf-8') for doc in text_test]
text_test = pd.DataFrame({"review": text_test, "pos": y_test})

In [7]:
text_train, y_train = reviews_train.data, reviews_train.target

print("Number of documents in train data: {}".format(len(text_train)))
print("Samples per class (train): {}".format(np.bincount(y_train)))

Number of documents in train data: 2000
Samples per class (train): [1000 1000]


In [8]:
text_train = [doc.replace(b"<br />", b" ").decode('utf-8') for doc in text_train]
text_train = pd.DataFrame({"review": text_train, "pos": y_train})

## Load Pre-Trained Models

In [9]:
import sys, os, pickle
import pandas as pd
from sklearn.externals import joblib

bow_model_path = os.getenv("DSX_PROJECT_DIR") + os.path.join("/models", os.getenv("DSX_MODEL_NAME","simple-bow"), os.getenv("DSX_MODEL_VERSION","2"),"model")

bow_classifier = joblib.load(open(bow_model_path, 'rb'))

## Train Topic Model Based Classifier

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

topic_train = pd.read_csv('../datasets/train-topic-probabilities.csv')
train_data = pd.read_csv('../datasets/train.csv')
train_data = train_data.join(topic_train)

In [11]:
topic_classifier = make_pipeline(FunctionTransformer(pd.DataFrame.get, kw_args={'key':['topic_' + str(i) for i in range(10)]}, validate=False),
                                 LogisticRegression())
scores = cross_val_score(topic_classifier, train_data, train_data.loc[:,'pos'], cv=5)
print("Mean cross-validation accuracy: {:.5f}".format(np.mean(scores)))

Mean cross-validation accuracy: 0.70200


## Merge Topic and BOW Classifiers  

In [12]:
ensemble_classifier = VotingClassifier([('bow', bow_classifier), ('topic', topic_classifier)], voting = 'soft')
scores_ensemble = cross_val_score(ensemble_classifier, train_data, train_data.loc[:,'pos'], cv=5)

/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

In [13]:
scores_bow = cross_val_score(bow_classifier, text_train, y_train, cv=5)

print("Mean cross-validation accuracy BOW: {:.5f}".format(np.mean(scores_bow)))
print("Mean cross-validation accuracy ensemble: {:.5f}".format(np.mean(scores_ensemble)))

Mean cross-validation accuracy BOW: 0.86650
Mean cross-validation accuracy ensemble: 0.86900
